In [53]:
import pandas as pd

df_raw = pd.read_csv("data/raw_data/lichen_moss_coverage_data.csv")
df_processed = df_raw

In [54]:
df_raw

,sample,angle,aspect,total_cover,moss_fraction,crustose,fruticose,foliose,species,deviation_north,solar_radiation
0,4,-31,260,8,3,1,0,1,"G_c,O_c,W_c,Bl_f",100,1749116.0
1,5,-6,298,3,5,1,0,1,"Bl_f,W_c,G_c,O_c",62,1702451.0
2,6,-16,351,17,29,1,0,0,"W_c,G_c",9,1746397.0
3,7,-12,129,1,0,1,0,0,"W_c,O_c",129,1626249.0
4,8,-17,55,6,11,1,0,0,"G_c,W_c,O_c",55,1541382.0
5,9,-17,321,48,4,1,0,1,"O_c,W_c,G_c,W_f",39,NaN
6,10,-23,324,38,89,1,0,1,"O_c,W_c,G_c,W_f,Bl_f",36,NaN
7,11,8,6,10,0,1,0,0,"G_c,O_c,W_c,GR_c",6,1562656.0
8,12,-9,286,1,0,1,0,0,"G_c,W_c",74,1733553.0
9,13,-10,12,39,10,1,1,1,"W_c,W_fr,W_f,G_c,GR_c",12,1659893.0


In [55]:
# the "total coverage" is the total amount of lichen AND moss covering the
# surface of a rock
# the "moss fraction" is the fraction OF THE TOTAL that is moss.
# so it's a percentage of a percentage (we recoreded it that way because
# it was methodologically easier)
# example: 10% total cover, 40% moss cover
# 40% of 10% = 4%
# therefore there is 4% moss, and the 6% remaining is lichen
# the code below processes our values into simply "moss coverage" and
# "lichen coverage"

# the cover values are %, so divide by 100 to get the true value
total_cover = df_raw['total_cover'] / 100
moss_fraction = df_raw['moss_fraction'] / 100
moss_cover = total_cover * moss_fraction
lichen_cover = total_cover - moss_cover

df_processed['moss_cover'] = moss_cover
df_processed['lichen_cover'] = lichen_cover


In [56]:
# convert the strings in "species" to lists of strings
species = pd.Series([species.split(',') for species in df_raw['species']])
df_processed['species'] = species


In [57]:
all_species = set()
for species_list in df_processed['species']:
    for unique_species in species_list:
        all_species.add(unique_species)

all_species


{'Bl_c',
 'Bl_f',
 'GR_c',
 'G_c',
 'O_c',
 'R_c',
 'W_c',
 'W_c ',
 'W_f',
 'W_fr',
 'Y_f'}

In [58]:
df_processed = df_processed.drop(columns=['moss_fraction'])

KeyError: "['Moss_fraction'] not found in axis"

In [ ]:
df_processed

,Sample,Angle,Aspect,Total_cover,Crustose,Fruticose,Foliose,Species,Deviation_north,Solar,moss_cover,lichen_cover
0,4,-31,260,8,1,0,1,"[G_c, O_c, W_c, Bl_f]",100,1749116.0,0.0024,0.0776
1,5,-6,298,3,1,0,1,"[Bl_f, W_c, G_c, O_c]",62,1702451.0,0.0015,0.0285
2,6,-16,351,17,1,0,0,"[W_c, G_c]",9,1746397.0,0.0493,0.1207
3,7,-12,129,1,1,0,0,"[W_c, O_c]",129,1626249.0,0.0000,0.0100
4,8,-17,55,6,1,0,0,"[G_c, W_c, O_c]",55,1541382.0,0.0066,0.0534
5,9,-17,321,48,1,0,1,"[O_c, W_c, G_c, W_f]",39,NaN,0.0192,0.4608
6,10,-23,324,38,1,0,1,"[O_c, W_c, G_c, W_f, Bl_f]",36,NaN,0.3382,0.0418
7,11,8,6,10,1,0,0,"[G_c, O_c, W_c, GR_c]",6,1562656.0,0.0000,0.1000
8,12,-9,286,1,1,0,0,"[G_c, W_c ]",74,1733553.0,0.0000,0.0100
9,13,-10,12,39,1,1,1,"[W_c, W_fr, W_f, G_c, GR_c]",12,1659893.0,0.0390,0.3510
